# Introduction to AI Agents

This notebook demonstrates how AI agents work, from basic function calling to a complete agentic loop.

## What is an Agent?

An AI agent is a system where an LLM can:
1. Decide which tools/functions to call
2. Execute those tools
3. Use the results to take further actions
4. Continue until the task is complete

In [1]:
# Setup
import os
import json
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## Step 1: Basic Python Functions

First, let's create some simple Python functions that our agent will be able to use.

In [2]:
def get_weather(location: str) -> str:
    """Get the current weather for a location."""
    # This is a mock function - in reality, you'd call a weather API
    weather_data = {
        "san francisco": "sunny, 72°F",
        "new york": "cloudy, 65°F",
        "london": "rainy, 58°F",
        "tokyo": "clear, 68°F"
    }
    location_lower = location.lower()
    return weather_data.get(location_lower, f"Weather data not available for {location}")

def calculate(operation: str, a: float, b: float) -> float:
    """Perform a mathematical calculation."""
    operations = {
        "add": a + b,
        "subtract": a - b,
        "multiply": a * b,
        "divide": a / b if b != 0 else "Error: Division by zero"
    }
    return operations.get(operation, "Error: Unknown operation")

def create_directory(directory_name: str) -> str:
    """Create a new directory."""
    try:
        os.makedirs(directory_name, exist_ok=True)
        return f"Successfully created directory: {directory_name}"
    except Exception as e:
        return f"Error creating directory: {str(e)}"

# Test our functions
print(get_weather("San Francisco"))
print(calculate("multiply", 15, 7))

sunny, 72°F
105


## Step 2: Define Tools for the LLM

Now we need to describe our functions in a format the LLM can understand. This is done using JSON schema.

In [3]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather for a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city name, e.g. San Francisco"
                    }
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate",
            "description": "Perform a mathematical calculation",
            "parameters": {
                "type": "object",
                "properties": {
                    "operation": {
                        "type": "string",
                        "enum": ["add", "subtract", "multiply", "divide"],
                        "description": "The mathematical operation to perform"
                    },
                    "a": {
                        "type": "number",
                        "description": "The first number"
                    },
                    "b": {
                        "type": "number",
                        "description": "The second number"
                    }
                },
                "required": ["operation", "a", "b"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "create_directory",
            "description": "Create a new directory on the file system",
            "parameters": {
                "type": "object",
                "properties": {
                    "directory_name": {
                        "type": "string",
                        "description": "The name of the directory to create"
                    }
                },
                "required": ["directory_name"]
            }
        }
    }
]

# Map function names to actual Python functions
available_functions = {
    "get_weather": get_weather,
    "calculate": calculate,
    "create_directory": create_directory
}

print(f"Defined {len(tools)} tools for the LLM")

Defined 3 tools for the LLM


## Step 3: Single Function Call Example

Let's see how the LLM decides to call a function and how we execute it.

In [4]:
# Create a simple query
messages = [
    {"role": "user", "content": "What's the weather in Tokyo?"}
]

# Send to OpenAI with tools available
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
    tool_choice="auto"  # Let the model decide when to use tools
)

response_message = response.choices[0].message
print("LLM Response:")
print(f"Content: {response_message.content}")
print(f"Tool calls: {response_message.tool_calls}")

LLM Response:
Content: None
Tool calls: [ChatCompletionMessageFunctionToolCall(id='call_RcQULoqAhIGEK56vBJ7sXbj2', function=Function(arguments='{"location":"Tokyo"}', name='get_weather'), type='function')]


The LLM decided to call a function! Let's execute it and send the result back.

In [ ]:
# Check if the model called a function
if response_message.tool_calls:
    # Add the assistant's response to messages
    messages.append(response_message)
    
    # Execute each tool call
    for tool_call in response_message.tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        
        print(f"\nCalling function: {function_name}")
        print(f"With arguments: {function_args}")
        
        # Get the actual function and call it
        function_to_call = available_functions[function_name]
        function_response = function_to_call(**function_args)
        
        print(f"Function returned: {function_response}")
        
        # Add the function response to messages
        messages.append({
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": str(function_response)
        })
    
    # Get the final response from the model
    final_response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    
    print(f"\nFinal Response: {final_response.choices[0].message.content}")
else:
    print("No function was called")

## Step 4: Multi-Step Agent Loop

Now let's create a complete agentic loop that can handle multiple tool calls until the task is complete.

In [5]:
def run_agent(user_query: str, max_iterations: int = 5) -> str:
    """
    Run an agentic loop that can make multiple tool calls.
    
    Args:
        user_query: The user's request
        max_iterations: Maximum number of back-and-forth iterations
    
    Returns:
        The final response from the agent
    """
    messages = [{"role": "user", "content": user_query}]
    
    print(f"User: {user_query}\n")
    
    for iteration in range(max_iterations):
        print(f"--- Iteration {iteration + 1} ---")
        
        # Get response from the model
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools,
            tool_choice="auto"
        )
        
        response_message = response.choices[0].message
        
        # If no tool calls, we're done
        if not response_message.tool_calls:
            print(f"Agent (final): {response_message.content}\n")
            return response_message.content
        
        # Add assistant's response to messages
        messages.append(response_message)
        
        # Execute all tool calls
        for tool_call in response_message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            print(f"Tool call: {function_name}({function_args})")
            
            # Execute the function
            function_to_call = available_functions[function_name]
            function_response = function_to_call(**function_args)
            
            print(f"Tool response: {function_response}")
            
            # Add function response to messages
            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": str(function_response)
            })
        
        print()
    
    return "Max iterations reached"

# Test with a simple query
result = run_agent("What's the weather in London?")
print(result)

User: What's the weather in London?

--- Iteration 1 ---
Tool call: get_weather({'location': 'London'})
Tool response: rainy, 58°F

--- Iteration 2 ---
Agent (final): The weather in London is rainy with a temperature of 58°F.

The weather in London is rainy with a temperature of 58°F.


## Step 5: Complex Multi-Tool Query

Now let's test with a query that requires multiple tool calls.

In [6]:
result = run_agent(
    """What's the weather in San Francisco and New York?
    Then multiply the numbers in their temperatures together."""
)

result

User: What's the weather in San Francisco and New York?
    Then multiply the numbers in their temperatures together.

--- Iteration 1 ---
Tool call: get_weather({'location': 'San Francisco'})
Tool response: sunny, 72°F
Tool call: get_weather({'location': 'New York'})
Tool response: cloudy, 65°F

--- Iteration 2 ---
Tool call: calculate({'operation': 'multiply', 'a': 72, 'b': 65})
Tool response: 4680

--- Iteration 3 ---
Agent (final): The weather in San Francisco is sunny with a temperature of 72°F, while in New York, it is cloudy with a temperature of 65°F. When you multiply the temperatures together, the result is 4680.



'The weather in San Francisco is sunny with a temperature of 72°F, while in New York, it is cloudy with a temperature of 65°F. When you multiply the temperatures together, the result is 4680.'

## Step 6: Action-Based Query

Let's test with a query that performs an action (creates a directory).

In [7]:
result = run_agent("Create a folder called 'agent-test-directory'")
result

User: Create a folder called 'agent-test-directory'

--- Iteration 1 ---
Tool call: create_directory({'directory_name': 'agent-test-directory'})
Tool response: Successfully created directory: agent-test-directory

--- Iteration 2 ---
Agent (final): The folder 'agent-test-directory' has been successfully created.



"The folder 'agent-test-directory' has been successfully created."

## Key Takeaways

1. **Function Calling**: The LLM can decide which functions to call based on the user's query
2. **Tool Execution**: We execute the requested functions and return results to the LLM
3. **Agentic Loop**: By repeatedly calling the LLM with tool results, we create an agent that can:
   - Break down complex tasks
   - Make multiple tool calls
   - Use results from one tool call to inform the next
   - Continue until the task is complete

This is the foundation of how AI agents work - they bridge the gap between language understanding and taking real actions!